# Parquet file (work in-progress)

In [1]:
import sys
import os

LINE_LENGTH = 200

In [2]:
def print_horizontal():
    """
    Simple method to print horizontal line
    :return: None
    """
    for i in range(LINE_LENGTH):
        sys.stdout.write('-')
    print("")

In [3]:
"""
try:
    # from pyspark import SparkContext
    from pyspark import SparkConf
    from pyspark import SQLContext

    print ("Successfully imported Spark Modules -- `SparkContext, SQLContext`")
    print_horizontal()
except ImportError as e:
    print ("Can not import Spark Modules", e)
    sys.exit(1)
"""    

Successfully imported Spark Modules -- `SparkContext, SQLContext`
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [4]:
conf = (SparkConf()
       .setAppName("Jon's Nation Cool App")
       .set("spark.executor.memory", "1g"))

sc = SparkContext(conf=conf)

sqlContext = SQLContext(sparkContext=sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[2]) created by <module> at /usr/local/lib/python2.7/dist-packages/IPython/utils/py3compat.py:289 

In [ ]:
parquetFile = sqlContext.read.parquet('./data/nation.plain.parquet')

In [ ]:
parquetFile.registerTempTable("parquetFile")

In [ ]:
nations_all_sql = sqlContext.sql("SELECT * FROM parquetFile")

In [ ]:
nations_all = nations_all_sql.map(lambda p: "Country: {0:15} Ipsum Comment: {1}".format(p.name, p.comment_col))

In [ ]:
print("All Nations and Comments -- `SELECT * FROM parquetFile`")
print_horizontal()
for nation in nations_all.collect():
    print(nation)

In [ ]:
nations_filtered_sql = sqlContext.sql("SELECT name FROM parquetFile WHERE name LIKE '%UNITED%'")

In [ ]:
nations_filtered = nations_filtered_sql.map(lambda p: "Country: {0:20}".format(p.name))

In [ ]:
print_horizontal()
print("Nations Filtered -- `SELECT name FROM parquetFile WHERE name LIKE '%UNITED%'`")
print_horizontal()
for nation in nations_filtered.collect():
    print(nation)

In [ ]:
print nations_filtered.collect()